In [ ]:
from google.colab import drive
drive.mount("/content/drive")
#drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
coupon_red=pd.read_csv('/content/drive/MyDrive/coupon_redempt.csv')
demo=pd.read_csv('/content/drive/MyDrive/hh_demographic.csv')

In [ ]:
coupon_red.head()
#coupon_red.count()

,household_key,DAY,COUPON_UPC,CAMPAIGN
0,1,421,10000085364,8
1,1,421,51700010076,8
2,1,427,54200000033,8
3,1,597,10000085476,18
4,1,597,54200029176,18


In [ ]:
demo

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16
...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498


In [ ]:
#Joining the dataframes 
demo.columns = map(str.lower, demo.columns)
coupon_red.columns = map(str.lower, coupon_red.columns)
hh_coupon=coupon_red.groupby('household_key')['coupon_upc'].count().reset_index()
hh_coupon.head()

,household_key,coupon_upc
0,1,5
1,8,1
2,13,21
3,14,3
4,18,8


In [ ]:
hh_coupon.count()

household_key    434
coupon_upc       434
dtype: int64

In [ ]:
#demo.join(hh_coupon,on=demo.household_key,lsuffix=)
coup_demo=demo.join(hh_coupon.set_index('household_key'),on='household_key',how='inner').reset_index()
coup_demo.drop(columns=['index'],inplace=True)
coup_demo

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key,coupon_upc
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,5
1,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,1
2,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,21
3,45-54,A,100-124K,Homeowner,2 Adults No Kids,2,None/Unknown,18,8
4,35-44,B,15-24K,Unknown,Single Female,1,None/Unknown,19,1
...,...,...,...,...,...,...,...,...,...
306,35-44,A,75-99K,Homeowner,2 Adults Kids,3,1,2467,5
307,45-54,B,35-49K,Homeowner,Single Female,1,None/Unknown,2488,11
308,35-44,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2489,28
309,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494,5


In [ ]:
trans=pd.read_csv('/content/drive/MyDrive/transaction_data.csv')
trans.columns = map(str.lower, trans.columns)

In [ ]:
trans.head()

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
#Finding the total visits made by customers to stores
vis=trans.groupby(['household_key'])['basket_id'].nunique().reset_index()
vis

,household_key,basket_id
0,1,86
1,2,45
2,3,47
3,4,30
4,5,40
...,...,...
2495,2496,63
2496,2497,221
2497,2498,172
2498,2499,90


In [ ]:
sales=trans.groupby('household_key')['sales_value'].sum().reset_index()
sales

,household_key,sales_value
0,1,4330.16
1,2,1954.34
2,3,2653.21
3,4,1200.11
4,5,779.06
...,...,...
2495,2496,4339.66
2496,2497,7111.98
2497,2498,2601.60
2498,2499,3394.07


In [ ]:
vis_sales=sales.merge(vis)
vis_sales.rename(columns={'basket_id':'total_visits'},inplace=True)

In [ ]:
vis_sales

,household_key,sales_value,total_visits
0,1,4330.16,86
1,2,1954.34,45
2,3,2653.21,47
3,4,1200.11,30
4,5,779.06,40
...,...,...,...
2495,2496,4339.66,63
2496,2497,7111.98,221
2497,2498,2601.60,172
2498,2499,3394.07,90


In [ ]:
demo_1=demo.join(vis_sales.set_index('household_key'),on='household_key',how='inner').reset_index().drop(columns='index')
demo_1

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key,sales_value,total_visits
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,4330.16,86
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,3400.05,59
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,5534.97,113
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,13190.92,275
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16,1512.02,98
...,...,...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494,2120.93,35
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496,4339.66,63
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497,7111.98,221
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498,2601.60,172


In [ ]:
df=demo_1.join(hh_coupon.set_index('household_key'),on='household_key',how='left').reset_index().drop(columns='index')
df

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key,sales_value,total_visits,coupon_upc
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,4330.16,86,5.0
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,3400.05,59,NaN
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,5534.97,113,1.0
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,13190.92,275,21.0
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16,1512.02,98,NaN
...,...,...,...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494,2120.93,35,5.0
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496,4339.66,63,11.0
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497,7111.98,221,NaN
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498,2601.60,172,NaN


In [ ]:

df['redemp']=df['coupon_upc'].apply(lambda x: 1 if x>=1 else 0)
df.drop(columns='coupon_upc',inplace=True)

In [ ]:
df

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key,sales_value,total_visits,redemp
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,4330.16,86,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,3400.05,59,0
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,5534.97,113,1
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,13190.92,275,1
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16,1512.02,98,0
...,...,...,...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494,2120.93,35,1
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496,4339.66,63,1
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497,7111.98,221,0
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498,2601.60,172,0


In [ ]:
df.isna().sum()

age_desc               0
marital_status_code    0
income_desc            0
homeowner_desc         0
hh_comp_desc           0
household_size_desc    0
kid_category_desc      0
household_key          0
sales_value            0
total_visits           0
redemp                 0
dtype: int64

In [ ]:
df.to_csv('/content/drive/MyDrive/data.csv')